In [ ]:
# Connecting the Python Code with the google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 18.7 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/73

In [ ]:
import pandas as pd
import re
import string
from bertopic import BERTopic
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Load pre-trained Word2Vec embeddings
word_vectors = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Emory/BMI NLP/Breast Cancer Data/GoogleNews-vectors-negative300.bin', binary=True)

# ITERATION 1: GPT-4 based list of uncertainty words
gpt4_concerns_keywords = [
    "uncertain", "doubtful", "ambiguous", "hesitant", "perplexed", "confused","questionable", "unsure", "indecisive", "bewildered", "vague", "puzzled",
    "skeptical", "inconclusive", "wavering", "distrustful", "baffled", "ambivalent", "hazy", "tentative"]

# ITERATION 2: Human Analysis of Random Posts + GPT assisted Keywords from Iteration 1
gpt4_human_concerns_iteration1_keywords = ["terrified", "scared", "nervous", "suspicion","uncertain","hesitant","perplexed" ,"clueless"
              , "worried", "alarmed", "anxiety", "unsure","puzzled", "skeptical", "baffled",
              "doubtful", "unaware" , "awkward", "paranoid", "myth", "confused", "embarrassed"]

# ITERATION 3: Final Set of Keywords (GPT 4 + Human Assisted Concerns Keywords) + Keywords related to prominent topic modeling of previous iteration
final_concerns_keywords = ["scared", "nervous","worried", "suspicion","uncertain","reluctant" ,"confused", "doubtful",
               "unsure","skeptical", "awkward", "insurance", "soy", "covid", "black", "bra","concerned"]

# Function to find similar words in a text
def find_similar_words(text, word_vectors, uncertainty_words, threshold=0.5):
    similar_words = []
    unc_words = []
    for word in text.lower().split():
        if word in word_vectors:
            word_embedding = word_vectors[word]
            for unc_word in uncertainty_words:
                if unc_word in word_vectors:
                    unc_embedding = word_vectors[unc_word]
                    similarity = cosine_similarity([word_embedding], [unc_embedding])[0][0]
                    if similarity > threshold:
                        similar_words.append(word)
                        similar_words.append(unc_word)
                        break
    return similar_words
# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/Emory/BMI NLP/Breast Cancer Data/Classification_ data/test_data_2.csv")
df['similar_uncertainty_words'] = df['text'].apply(lambda x: find_similar_words(x, word_vectors, final_concerns_keywords))
df.to_csv('/content/drive/MyDrive/Emory/BMI NLP/Breast Cancer Data/CSV Files/concerns_test_data2.csv', encoding='utf-8', index=False)

In [ ]:
# Dataset Statistic
df_full = pd.read_csv('/content/drive/MyDrive/Emory/BMI NLP/Breast Cancer Data/CSV Files/concerns_test_data2.csv.csv')
print("Shape of df_full: ", df_full.shape)
df_full = df_full[df_full['similar_uncertainty_words']!='[]']
df_full.dropna(inplace=True)
print("Shape of df_full with keywords matched: ", df_full.shape)

Shape of df_full:  (83695, 5)
Shape of df_full with keywords matched:  (7863, 5)


In [ ]:
# Pre-load the stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_emojis(text):
    # Regular expression to match emojis
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
def preprocess_text_optimized(text):
    # text = remove_emojis(text)
    text = text.lower()
    # Remove <word> patterns
    text = re.sub(r'<[^>]+>', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.replace('@', '').replace('\n', '')
    tokens = word_tokenize(text)
    # Filter out stopwords
    filtered_text = [word for word in tokens if word not in STOPWORDS]
    return ' '.join(filtered_text)

# Apply the preprocess function to the DataFrame column
df_full['text'] = df_full['text'].apply(preprocess_text_optimized)
tweets = df_full['text'].to_list()
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(tweets)

2024-03-31 02:59:13,121 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/246 [00:00<?, ?it/s]

2024-03-31 02:59:52,219 - BERTopic - Embedding - Completed ✓
2024-03-31 02:59:52,221 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-31 03:00:04,163 - BERTopic - Dimensionality - Completed ✓
2024-03-31 03:00:04,165 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-31 03:00:04,585 - BERTopic - Cluster - Completed ✓
2024-03-31 03:00:04,591 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-31 03:00:05,032 - BERTopic - Representation - Completed ✓


In [ ]:
# Visualize the Concerns/Fears amongst Breast Cancer Twitter Cohort
topic_model.visualize_barchart(top_n_topics=50)